In [1]:
#--------------------------------------------
# QOSF Mentorship Screening Task Submission
# Task 2 
# Bach Bao
# bao.bachbbace12@hcmut.edu.vn
# Sep-2020
#--------------------------------------------

In [2]:
import qiskit
import math
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer, execute
from qiskit.providers.aer import QasmSimulator
from qiskit.aqua.components.optimizers import COBYLA,SPSA
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise import QuantumError, ReadoutError
from qiskit.test.mock import FakeVigo
from qiskit.tools.monitor import job_monitor
from scipy.optimize import minimize

In [3]:
# Function to create circuit cotains of 2 qubits and gate Rx, Ry, CNOT

def create_circuit(parameters):
    
    qctl = QuantumRegister(2)
    qc = ClassicalRegister(2)
    circ = QuantumCircuit(qctl, qc)

    circ.rx(parameters[0],qctl[0])
    circ.ry(parameters[1],qctl[0]) # To create H gate (1 Rx and 1 Ry) for qubit 0
    
    circ.rx(parameters[2],qctl[1]) # To create X gate (1 Rx) for qubit 1
    
    circ.cx(qctl[0],qctl[1]) # To entangle qubit 0 and 1
    
    circ.measure(qctl,qc)
    
    return circ

In [4]:
# Function to calculate cost function to bring to the optimizer

def calculate_cost_function(parameters):
    
    phase=0 
    probability={}
    times=1000
    simulator = QasmSimulator() # Define the simulator
     
    # Create circuit
    circ = create_circuit(parameters)
    
    # Create a noise model using FakeVigo as device backend 
    device_backend = FakeVigo()
    coupling_map = device_backend.configuration().coupling_map
    noise_model = NoiseModel.from_backend(device_backend)
    
    # Simulate the circuit for n times with the device backend noise model
    job = execute(circ, simulator,
                basis_gates = noise_model.basis_gates,
                noise_model = noise_model,
                shots = times)    
    counts = job.result().get_counts()
    
    # Fill the value
    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0
    
    # Get the probability after simulating
    for i in counts.keys():
        probability[i] = counts[i]/times
    
    # After going through Rx and Ry, the relative phase of |1> is rev_phase (matrix multiplication)
    rev_phase= (np.cos(parameters[0]/2)*np.sin(parameters[1]/2) - 1j*np.sin(parameters[0]/2)*np.cos(parameters[1]/2)) / (np.cos(parameters[0]/2)*np.cos(parameters[1]/2) + 1j*np.sin(parameters[0]/2)*np.sin(parameters[1]/2))
    
    #rev_phase is equal to e^(i*phase) and we want to achieve rev_phase=1 so phase=0
    
    print (probability,rev_phase)
    # Complete the cost_function (least square)
    return ((probability['00'])**2 + (probability['01']-1/2)**2 + (probability['10']-1/2)**2 + (probability['11'])**2)+ (abs(rev_phase-np.exp(1j*phase)))**2  

In [11]:
# Using Powell method to optimize the outcome 
outcome = minimize(calculate_cost_function, x0=[0,0,0], method="POWELL", options={'maxiter':400})
print(outcome)

{'00': 0.984, '01': 0.009, '10': 0.007, '11': 0.0} 0j
{'00': 0.974, '01': 0.012, '10': 0.014, '11': 0.0} 0j
{'00': 0.765, '01': 0.01, '10': 0.007, '11': 0.218} -0.5463024898437905j
{'00': 0.067, '01': 0.028, '10': 0.023, '11': 0.882} -3.7323503047403657j
{'00': 0.792, '01': 0.006, '10': 0.011, '11': 0.191} -0.5463024898437905j
{'00': 0.459, '01': 0.026, '10': 0.015, '11': 0.5} -1.048389550684371j
{'00': 0.891, '01': 0.016, '10': 0.008, '11': 0.085} -0.31924398056579173j
{'00': 0.875, '01': 0.012, '10': 0.011, '11': 0.102} -0.3620884475709664j
{'00': 0.909, '01': 0.006, '10': 0.008, '11': 0.077} -0.2888288193606052j
{'00': 0.872, '01': 0.007, '10': 0.012, '11': 0.109} -0.3353310055014342j
{'00': 0.896, '01': 0.006, '10': 0.002, '11': 0.096} -0.33893422015580993j
{'00': 0.897, '01': 0.017, '10': 0.005, '11': 0.081} -0.3317356010498449j
{'00': 0.881, '01': 0.01, '10': 0.007, '11': 0.102} -0.3353310055014342j
{'00': 0.692, '01': 0.016, '10': 0.012, '11': 0.28} (0.4691288108333316-0.4212717

{'00': 0.036, '01': 0.467, '10': 0.486, '11': 0.011} (0.8608978866311685-0.3760958157039484j)
{'00': 0.027, '01': 0.501, '10': 0.463, '11': 0.009} (0.9828078210735431+0.2064069557452074j)
{'00': 0.038, '01': 0.5, '10': 0.448, '11': 0.014} (0.9800455391419708-0.0019233174402307415j)
{'00': 0.029, '01': 0.493, '10': 0.464, '11': 0.014} (0.9755026157895377-0.035310280983900516j)
{'00': 0.027, '01': 0.454, '10': 0.508, '11': 0.011} (0.9773804207587461-0.022494496695486456j)
{'00': 0.03, '01': 0.49, '10': 0.466, '11': 0.014} (0.9767175359867099-0.027150561534295715j)
{'00': 0.03, '01': 0.476, '10': 0.481, '11': 0.013} (0.9769467048858402-0.02555831989243114j)
{'00': 0.046, '01': 0.477, '10': 0.455, '11': 0.022} (0.977113455832329-0.024388330010252143j)
{'00': 0.029, '01': 0.475, '10': 0.479, '11': 0.017} (0.976859463410573-0.02616658006576566j)
{'00': 0.031, '01': 0.443, '10': 0.503, '11': 0.023} (0.9770105564535916-0.025111465445754787j)
{'00': 0.034, '01': 0.468, '10': 0.483, '11': 0.015}

{'00': 0.028, '01': 0.198, '10': 0.77, '11': 0.004} (0.5003434049486375-0.017617349097791064j)
{'00': 0.03, '01': 0.649, '10': 0.312, '11': 0.009} (1.439775603774462-0.04331362282613202j)
{'00': 0.024, '01': 0.447, '10': 0.522, '11': 0.007} (0.9211060737055528-0.026047575942990668j)
{'00': 0.039, '01': 0.462, '10': 0.489, '11': 0.01} (0.9812271967561391-0.027659875737767226j)
{'00': 0.032, '01': 0.466, '10': 0.496, '11': 0.006} (0.9663047604341459-0.0272501777786256j)
{'00': 0.029, '01': 0.45, '10': 0.512, '11': 0.009} (0.9728728005909641-0.0274297302722353j)
{'00': 0.025, '01': 0.495, '10': 0.469, '11': 0.011} (0.9775001200080355-0.027556959555436803j)
{'00': 0.025, '01': 0.452, '10': 0.515, '11': 0.008} (0.9789220984647211-0.027596178559858048j)
{'00': 0.024, '01': 0.468, '10': 0.5, '11': 0.008} (0.9766222983240881-0.02753277719749001j)
{'00': 0.021, '01': 0.475, '10': 0.495, '11': 0.009} (0.9770012954942178-0.027543215198829272j)
{'00': 0.029, '01': 0.469, '10': 0.492, '11': 0.01} (

{'00': 0.026, '01': 0.478, '10': 0.486, '11': 0.01} (0.977206539592327-0.026106573132656j)
{'00': 0.033, '01': 0.447, '10': 0.514, '11': 0.006} (0.9772747092100689-0.025631307367094118j)
{'00': 0.029, '01': 0.473, '10': 0.486, '11': 0.012} (0.977234676885778-0.025910599323299838j)
{'00': 0.033, '01': 0.459, '10': 0.497, '11': 0.011} (0.977260706624533-0.02572906163840644j)
{'00': 0.026, '01': 0.461, '10': 0.505, '11': 0.008} (0.9772454144408154-0.025835741352345128j)
{'00': 0.026, '01': 0.496, '10': 0.468, '11': 0.01} (0.9772553556411027-0.025766399790449115j)
{'00': 0.032, '01': 0.464, '10': 0.498, '11': 0.006} (0.9772495143722465-0.02580714776773028j)
{'00': 0.027, '01': 0.476, '10': 0.49, '11': 0.007} (0.9772533113867419-0.025780661599314084j)
{'00': 0.031, '01': 0.466, '10': 0.498, '11': 0.005} (0.9772510801951679-0.025796225934442998j)
{'00': 0.035, '01': 0.482, '10': 0.478, '11': 0.005} (0.9772525304984672-0.025786109111433034j)
{'00': 0.025, '01': 0.48, '10': 0.49, '11': 0.005} 

{'00': 0.024, '01': 0.485, '10': 0.48, '11': 0.011} (0.9775963543139784-0.025798357541326276j)
{'00': 0.034, '01': 0.496, '10': 0.463, '11': 0.007} (0.9775884838762507-0.02579815448078333j)
{'00': 0.027, '01': 0.484, '10': 0.481, '11': 0.008} (0.9775918828067471-0.0257982421743928j)
{'00': 0.023, '01': 0.479, '10': 0.483, '11': 0.015} (0.9779949550735854-0.02295807553033954j)
{'00': 0.027, '01': 0.473, '10': 0.488, '11': 0.012} (0.9786303644940673-0.018358198083632732j)
{'00': 0.032, '01': 0.483, '10': 0.477, '11': 0.008} (0.9779949550735854-0.02295807553033954j)
{'00': 0.032, '01': 0.454, '10': 0.509, '11': 0.005} (0.9782401101242915-0.021201716088325905j)
{'00': 0.029, '01': 0.478, '10': 0.485, '11': 0.008} (0.9778419287068744-0.02404316890763089j)
{'00': 0.027, '01': 0.491, '10': 0.476, '11': 0.006} (0.9778314040431622-0.024117487469463253j)
{'00': 0.038, '01': 0.465, '10': 0.493, '11': 0.004} (0.9777402419478558-0.024759565077748003j)
{'00': 0.028, '01': 0.478, '10': 0.486, '11': 0